In [ ]:
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np

# --- 실험군 설정 (A3-A ~ A3-D 중 택1) ---
experiment_group = 'A3-A'  # 'A3-A', 'A3-B', 'A3-C', 'A3-D'

# --- 초기 상태 설정 ---
def prepare_initial_state(circuit, qubit):
    if experiment_group in ['A3-A', 'A3-C']:
        pass  # |0⟩ 상태
    else:
        circuit.h(qubit)  # |+⟩ 상태

# --- 분기 구조 회로 정의 ---
def apply_branch_structure(circuit, control, target):
    circuit.h(control)
    circuit.cx(control, target)

# --- 실험 회로 구성 ---
repetitions = 3
backend = Aer.get_backend('qasm_simulator')
shots = 1024
all_counts = []

for rep in range(repetitions):
    qc = QuantumCircuit(2, 2)

    # 초기 상태 준비
    prepare_initial_state(qc, 0)

    # 분기 구조 적용
    if experiment_group in ['A3-C', 'A3-D']:
        if np.random.rand() > 0.5:
            qc.x(0)  # 랜덤 분기 조건 조작
    apply_branch_structure(qc, 0, 1)

    # 측정 설정
    if experiment_group in ['A3-B']:  # 반복별 개별 측정
        qc.measure([0, 1], [0, 1])
        job = backend.run(qc, shots=shots)
        counts = job.result().get_counts()
        all_counts.append(counts)

    elif experiment_group in ['A3-D']:  # 반복 평균 측정
        qc.measure([0, 1], [0, 1])
        job = backend.run(qc, shots=shots)
        counts = job.result().get_counts()
        all_counts.append(counts)

    elif experiment_group in ['A3-A', 'A3-C']:
        if rep == repetitions - 1:  # 마지막 반복에만 측정
            qc.measure([0, 1], [0, 1])
            job = backend.run(qc, shots=shots)
            counts = job.result().get_counts()
            all_counts.append(counts)

# --- 결과 시각화 ---
def merge_counts(list_of_counts):
    merged = {}
    for counts in list_of_counts:
        for key, val in counts.items():
            merged[key] = merged.get(key, 0) + val
    return merged

if experiment_group in ['A3-B']:
    for i, counts in enumerate(all_counts):
        fig = plot_histogram(counts, title=f'{experiment_group} - Repetition {i+1}')
        fig.savefig(f'{experiment_group}_rep{i+1}.png')
        plt.close(fig)
else:
    if experiment_group == 'A3-D':
        merged = merge_counts(all_counts)
    else:
        merged = all_counts[0]

    fig = plot_histogram(merged, title=f'{experiment_group} - Result')
    fig.savefig(f'{experiment_group}_result.png')
    plt.close(fig)



In [ ]:
#정량 분석 코드 삽입 :

from qiskit import QuantumCircuit
from qiskit_aer import Aer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.distance import cosine

# --- 실험군 설정 (A3-A ~ A3-D 중 택1) ---
experiment_group = 'A3-A'  # 'A3-A', 'A3-B', 'A3-C', 'A3-D'

# --- 정량 지표 함수 정의 ---
def shannon_entropy(counts):
    total = sum(counts.values())
    probs = [v / total for v in counts.values()]
    return -sum(p * np.log2(p) for p in probs if p > 0)

def distribution_vector(counts, keys):
    total = sum(counts.values())
    return np.array([counts.get(k, 0) / total for k in keys])

# --- 초기 상태 설정 ---
def prepare_initial_state(circuit, qubit):
    if experiment_group in ['A3-A', 'A3-C']:
        pass  # |0⟩ 상태
    else:
        circuit.h(qubit)  # |+⟩ 상태

# --- 분기 구조 회로 정의 ---
def apply_branch_structure(circuit, control, target):
    circuit.h(control)
    circuit.cx(control, target)

# --- 실험 회로 구성 ---
repetitions = 3
backend = Aer.get_backend('qasm_simulator')
shots = 1024
all_counts = []

for rep in range(repetitions):
    qc = QuantumCircuit(2, 2)

    # 초기 상태 준비
    prepare_initial_state(qc, 0)

    # 분기 구조 적용
    if experiment_group in ['A3-C', 'A3-D']:
        if np.random.rand() > 0.5:
            qc.x(0)  # 랜덤 분기 조건 조작
    apply_branch_structure(qc, 0, 1)

    # 측정 설정
    if experiment_group in ['A3-B', 'A3-D']:
        qc.measure([0, 1], [0, 1])
        job = backend.run(qc, shots=shots)
        counts = job.result().get_counts()
        all_counts.append(counts)

    elif experiment_group in ['A3-A', 'A3-C']:
        if rep == repetitions - 1:
            qc.measure([0, 1], [0, 1])
            job = backend.run(qc, shots=shots)
            counts = job.result().get_counts()
            all_counts.append(counts)

# --- 결과 시각화 ---
def merge_counts(list_of_counts):
    merged = {}
    for counts in list_of_counts:
        for key, val in counts.items():
            merged[key] = merged.get(key, 0) + val
    return merged

# --- 결과 처리 및 저장 ---
if experiment_group == 'A3-B':
    for i, counts in enumerate(all_counts):
        fig = plot_histogram(counts, title=f'{experiment_group} - Repetition {i+1}')
        fig.savefig(f'{experiment_group}_rep{i+1}.png')
        plt.close(fig)
else:
    if experiment_group == 'A3-D':
        merged = merge_counts(all_counts)
    else:
        merged = all_counts[0]
    fig = plot_histogram(merged, title=f'{experiment_group} - Result')
    fig.savefig(f'{experiment_group}_result.png')
    plt.close(fig)

# --- 정량 지표 분석 ---
entropies = [shannon_entropy(c) for c in all_counts]

# 엔트로피 플롯
plt.plot(range(1, len(entropies)+1), entropies, marker='o')
plt.title(f'{experiment_group} - Shannon Entropy Across Repeats')
plt.xlabel("Repetition")
plt.ylabel("Entropy")
plt.grid(True)
plt.savefig("entropy_plot.png")
plt.show()

# 유사도 로그
with open("similarity_log.txt", "w", encoding="utf-8") as f:
    f.write(f"{experiment_group} - Cosine Similarity Log\n-----------------------------\n")
    for i in range(len(all_counts) - 1):
        c1, c2 = all_counts[i], all_counts[i+1]
        keys = sorted(set(c1) | set(c2))
        v1 = distribution_vector(c1, keys)
        v2 = distribution_vector(c2, keys)
        sim = 1 - cosine(v1, v2)
        f.write(f"Repeat {i+1} vs {i+2}: {sim:.4f}\n")
